In [ ]:
# remove LLM-detected errors from original VariErr

import json

varierr_file = '../varierr.json'
model_file = '../llama33_70b_explanation_with_validation_count.jsonl'
output_file = '../varierr_without_70b.json'

with open(varierr_file, 'r', encoding='utf-8') as f:
    varierr_data = {json.loads(line)['id']: json.loads(line) for line in f}

with open(model_file, 'r', encoding='utf-8') as f:
    model_data = {json.loads(line)['id']: json.loads(line) for line in f}
label_map = {'e': 'entailment', 'n': 'neutral', 'c': 'contradiction'}

merged = []

for uid, var_entry in varierr_data.items():
    model_entry = model_data.get(uid, {})

    var_entry.pop('entailment', None)
    var_entry.pop('contradiction', None)
    var_entry.pop('neutral', None)
    var_entry.pop('idk', None)

    if 'error' in model_entry:
        error_raw = model_entry['error']
        error_mapped = [label_map.get(lbl, lbl) for lbl in error_raw]
        var_entry['error_llm'] = error_mapped
    if 'not_validated_exp' in model_entry:
        var_entry['not_validated_exp_llm'] = model_entry['not_validated_exp']

    original_labels = set(var_entry.get('label_set_round_1', []))
    error_labels = set(var_entry.get('error_llm', []))
    label_set_llm = sorted(original_labels - error_labels)

    var_entry['label_set_llm'] = label_set_llm
    merged.append(var_entry)

with open(output_file, 'w', encoding='utf-8') as out:
    for item in merged:
        json.dump(item, out, ensure_ascii=False)
        out.write('\n')

print(f"Done.")


In [ ]:
import json
from collections import Counter

input_path = "../varierr_without_70b.json"
output_path = "../varierr_without_70b_dist.json"

def convert_label_list_to_dist(label_list):
    counter = Counter(label_list or [])
    e = counter.get("entailment", 0)
    n = counter.get("neutral", 0)
    c = counter.get("contradiction", 0)

    total = e + n + c
    if total == 0:
        return [0.0, 0.0, 0.0]
    return [e / total, n / total, c / total] 

with open(input_path, "r") as infile, \
     open(output_path, "w") as outfile:

    for line in infile:
        item = json.loads(line)

        label_list = item.get("label_set_llm")
        if label_list:
            item["label"] = convert_label_list_to_dist(label_list)
        else:
            item["label"] = [0.0, 0.0, 0.0]

        json.dump(item, outfile)
        outfile.write("\n")

print("Done")


In [ ]:
## clean

import json
from pathlib import Path
from tqdm import tqdm

def normalize_label_dist(chaos_dict):
   
    label_order = ["entailment", "neutral", "contradiction"]
    values = [(chaos_dict.get(k) or 0.0) for k in label_order] 
    total = sum(values)
    if total == 0:
        return [0.0] * 3
    return [v / total for v in values]


def process_file(input_path, output_path):
    with open(input_path, "r", encoding="utf-8") as fin, open(output_path, "w", encoding="utf-8") as fout:
        for line in tqdm(fin, desc=f"Processing {input_path}"):
            raw = json.loads(line)
            out = {
                "uid": raw.get("id", raw.get("uid")),
                "premise": raw.get("context"),
                "hypothesis": raw.get("statement"),
                "label": raw.get("label")
            }
            fout.write(json.dumps(out, ensure_ascii=False) + "\n")

if __name__ == "__main__":
    input_path = "../varierr_without_70b_dist.json"
    output_path = "../varierr_without_70b_cleaned.json"
    process_file(input_path, output_path)
